Meeting notes:

##### Make diag.bib update code (notebook)
- Compare titles of bib and semantic scholar
- Match
- If match:
    - Add semantic scholar ID to diag bib
    - Update gscites field from bib entry
- If no match:
    - Check best matches, if DOI and Arxiv --> chose DOI 
    - Define right fields (look at code James) + citations + semantic scolar ID 
    - Add 'automatically added' field so we can trace back automation errors more easilty
    - Add to bib   
    
##### Make sript that checks ss ids and adds new ones to diag bib
    
    
##### Make script that can be used by DIAG members
- Add doi/arxiv
- if DOI check if Arxiv exists, if so --> remove Arxiv
- Sort right fields from DOI --> add fields
- Search for semantic scholar ID
- Print fields so user can check it
- Add to bib
- Manual commit push

##### Make script that checks all semantic scholar IDs and updates gscites
- github action overnight?

##### Note: 
- requests limit, 100 requests per 5 mins for ss 
    - 100 papers at once = 1 request
- google doc: 
    - https://docs.google.com/document/d/1uFyh5Z-wieksq_4Bf8Br7wHpN-buHECunS0VurYnlvg/edit


# Libraries

In [1]:
import os
import numpy as np
import pandas as pd

import requests
# from bibreader import parse_bibtex_file, get_bib_blocks
from bib_handling_code.processbib import read_bibfile
from bib_handling_code.processbib import save_to_file
from difflib import SequenceMatcher

# Example from url giving 100 papers from one of Bram's semantic scolar IDs

In [2]:
staff_id = 123637526 #8038506
staff_ids = [123637526]

In [3]:
url=f'https://api.semanticscholar.org/graph/v1/author/{staff_id}/papers?fields=year,title,authors,externalIds,citationCount&limit=100'

r = requests.get(url)
ss_staff_data = r.json()['data']

In [4]:
for i in ss_staff_data:
    if i['paperId'] == 'c70d5169b5c69171cc2ed3ec40eebda0630988d9':
        display(i)

{'paperId': 'c70d5169b5c69171cc2ed3ec40eebda0630988d9',
 'externalIds': {'DOI': '10.1038/s41598-020-62148-y', 'CorpusId': 257025991},
 'title': 'Computer aided detection of tuberculosis on chest radiographs: An evaluation of the CAD4TB v6 system',
 'year': 2020,
 'citationCount': 0,
 'authors': [{'authorId': '35730362', 'name': 'K. Murphy'},
  {'authorId': '6214077', 'name': 'S. Habib'},
  {'authorId': '40347668', 'name': 'S. M. Zaidi'},
  {'authorId': '39726137', 'name': 'S. Khowaja'},
  {'authorId': '2208892980', 'name': 'Aamir Khan'},
  {'authorId': '144861586', 'name': 'J. Melendez'},
  {'authorId': '152867460', 'name': 'E. Scholten'},
  {'authorId': '81054906', 'name': 'Farhan Amad'},
  {'authorId': '3504603', 'name': 'S. Schalekamp'},
  {'authorId': '40442474', 'name': 'Maurits Verhagen'},
  {'authorId': '38557856', 'name': 'R. Philipsen'},
  {'authorId': '2074941083', 'name': 'Annet Meijers'},
  {'authorId': '123637526', 'name': 'B. van Ginneken'}]}

# Function to get data from DOI/ArXiv

In [5]:
def get_doi_csl(doi):
    response = requests.get(
        f"https://doi.org/{doi}",
        headers={"Accept": "application/vnd.citationstyles.csl+json"},
        timeout=5,
    )
    response.raise_for_status()
    return response.json()

## Example

In [6]:
get_doi_csl('10.48550/arXiv.2302.01738').keys()

dict_keys(['type', 'id', 'categories', 'author', 'issued', 'abstract', 'DOI', 'publisher', 'title', 'URL', 'copyright', 'version'])

In [7]:
ss_staff_data[2]['externalIds']

{'DOI': '10.1007/s00330-023-09826-3',
 'CorpusId': 259200504,
 'PubMed': '37338552'}

In [8]:
staff_id = 8038506
url=f'https://api.semanticscholar.org/graph/v1/author/{staff_id}/papers?fields=year,title,authors,externalIds,citationCount&limit=100'

r = requests.get(url)
ss_staff_data = r.json()['data']

In [9]:
found_dois = []

In [10]:
found_arxiv_doi = False
found_regular_doi = False

In [11]:
#ArXiv
for ss_staff_entry in ss_staff_data:
    if 'DOI' in ss_staff_entry['externalIds']:
        if 'arXiv' in ss_staff_entry['externalIds']['DOI']:
            arxiv_doi = ss_staff_entry['externalIds']['DOI']
            found_arxiv_doi = True
    if found_arxiv_doi:
        break
#DOI
for ss_staff_entry in ss_staff_data:
    if 'DOI' in ss_staff_entry['externalIds']:
        if 'arXiv' not in ss_staff_entry['externalIds']['DOI']:
            regular_doi = ss_staff_entry['externalIds']['DOI']
            
            found_regular_doi = True
    if found_regular_doi:
        break

print(arxiv_doi)
print(regular_doi)

10.48550/arXiv.2302.01790
10.1016/j.media.2023.102755


In [12]:
arxiv_doi_data = get_doi_csl(arxiv_doi)
regular_doi_data = get_doi_csl(regular_doi)

In [13]:
for ss in ss_staff_data:
    if 'type' in ss:
        print(ss['type'])

In [14]:
[print(key) for key in arxiv_doi_data.keys()];

type
id
categories
author
issued
abstract
DOI
publisher
title
URL
copyright
version


In [15]:
arxiv_doi_data

{'type': 'article',
 'id': 'https://doi.org/10.48550/arxiv.2302.01790',
 'categories': ['Computer Vision and Pattern Recognition (cs.CV)',
  'FOS: Computer and information sciences',
  'FOS: Computer and information sciences'],
 'author': [{'family': 'Reinke', 'given': 'Annika'},
  {'family': 'Tizabi', 'given': 'Minu D.'},
  {'family': 'Baumgartner', 'given': 'Michael'},
  {'family': 'Eisenmann', 'given': 'Matthias'},
  {'family': 'Heckmann-Nötzel', 'given': 'Doreen'},
  {'family': 'Kavur', 'given': 'A. Emre'},
  {'family': 'Rädsch', 'given': 'Tim'},
  {'family': 'Sudre', 'given': 'Carole H.'},
  {'family': 'Acion', 'given': 'Laura'},
  {'family': 'Antonelli', 'given': 'Michela'},
  {'family': 'Arbel', 'given': 'Tal'},
  {'family': 'Bakas', 'given': 'Spyridon'},
  {'family': 'Benis', 'given': 'Arriel'},
  {'family': 'Blaschko', 'given': 'Matthew'},
  {'family': 'Büttner', 'given': 'Florian'},
  {'family': 'Cardoso', 'given': 'M. Jorge'},
  {'family': 'Cheplygina', 'given': 'Veronika'},

In [16]:
[print(key) for key in regular_doi_data.keys()];

indexed
reference-count
publisher
license
content-domain
published-print
DOI
type
created
page
update-policy
source
is-referenced-by-count
title
prefix
volume
author
member
reference
container-title
original-title
language
link
deposited
score
resource
subtitle
short-title
issued
references-count
alternative-id
URL
relation
ISSN
subject
container-title-short
published
assertion
article-number


In [17]:
regular_doi_data

{'indexed': {'date-parts': [[2023, 8, 15]],
  'date-time': '2023-08-15T22:05:10Z',
  'timestamp': 1692137110850},
 'reference-count': 38,
 'publisher': 'Elsevier BV',
 'license': [{'start': {'date-parts': [[2023, 4, 1]],
    'date-time': '2023-04-01T00:00:00Z',
    'timestamp': 1680307200000},
   'content-version': 'tdm',
   'delay-in-days': 0,
   'URL': 'https://www.elsevier.com/tdm/userlicense/1.0/'},
  {'start': {'date-parts': [[2023, 2, 6]],
    'date-time': '2023-02-06T00:00:00Z',
    'timestamp': 1675641600000},
   'content-version': 'vor',
   'delay-in-days': 0,
   'URL': 'http://creativecommons.org/licenses/by/4.0/'}],
 'content-domain': {'domain': ['elsevier.com', 'sciencedirect.com'],
  'crossmark-restriction': True},
 'published-print': {'date-parts': [[2023, 4]]},
 'DOI': '10.1016/j.media.2023.102755',
 'type': 'journal-article',
 'created': {'date-parts': [[2023, 1, 25]],
  'date-time': '2023-01-25T12:21:08Z',
  'timestamp': 1674649268000},
 'page': '102755',
 'update-poli

---
---
---

#####################################
#  FINAL
#####################################

---
---
---


# Import diag.bib

In [18]:
path_diag_bib = os.path.join('..', 'diag.bib')
diag_bib_raw = read_bibfile(None, path_diag_bib) # I changed the code in such a way that IF I give a second argument, it uses the second argument as a full path

In [19]:
len(diag_bib_raw)

2822

In [20]:
diag_bib_raw[-1].fields

{'author': '{Philipp, Lena}',
 'title': '{Body Composition Assessment in 3D CT Images}',
 'abstract': '{Body composition as a diagnostic and prognostic biomarker is gaining importance in various medical fields such as oncology. Therefore, accurate quantification methods are necessary, such as analyzing CT images. While several studies introduced deep learning approaches to automatically segment a single slice, quantifying body composition in 3D remains understudied due to the high required annotation effort. This thesis explores the use of annotation-efficient strategies in developing a body composition segmentation model for the abdomen and pelvis. To address this, a fine-tuning strategy was proposed to optimize the annotation process and extend the model’s generalization performance trained with L3 slices to the entire abdomen and pelvis. Moreover, a self-supervised pre-training using a contrastive loss was employed to leverage unlabeled data. The goal was to efficiently use the anno

In [21]:
bib_data = []
for bib_entry in diag_bib_raw:
    bibkey = bib_entry.key
    bib_type = bib_entry.type
    
    bib_authors = None
    bib_title = None
    bib_doi = None
    bib_gscites = None
    bib_journal = None
    bib_year = None
    
    if 'author' in bib_entry.fields:
        bib_authors = bib_entry.fields['author'].strip('{ }')
    if 'title' in bib_entry.fields:
        bib_title = bib_entry.fields['title'].strip('{ }')
    if 'doi' in bib_entry.fields:
        bib_doi = bib_entry.fields['doi'].strip('{ }')
    if 'gscites' in bib_entry.fields:
        bib_gscites = bib_entry.fields['gscites'].strip('{ }')
    if 'journal' in bib_entry.fields:
        bib_journal = bib_entry.fields['journal'].strip('{ }')
    if 'year' in bib_entry.fields:
        bib_year = bib_entry.fields['year'].strip('{ }')
    
    bib_data.append([bibkey, bib_type, bib_title, bib_authors, bib_doi, bib_gscites, bib_journal, bib_year])
bib_columns = ['bibkey', 'type', 'title', 'authors', 'doi', 'gs_citations', 'journal', 'year']
df_bib_data = pd.DataFrame(bib_data, columns=bib_columns)

In [22]:
df_bib_data.to_csv('script_data/temp_diag_bib_summary.csv', index=False)

In [23]:
df_bib_data.tail()

,bibkey,type,title,authors,doi,gs_citations,journal,year
2817,Thij23,article,Radiomics based automated quality assessment f...,"Thijssen, Linda C.P. and de Rooij, Maarten and...",10.1016/j.ejrad.2023.110928,None,European Journal of Radiology,2023
2818,Venk23,article,Prior CT Improves Deep Learning for Malignancy...,"Venkadesh, Kiran Vaidhya and Aleef, Tajwar Abr...",10.1148/radiol.223308,None,Radiology,2023
2819,Leeu23a,article,Clinical use of artificial intelligence produc...,"van Leeuwen, Kicky G. and de Rooij, Maarten an...",10.1007/s00330-023-09991-5,None,European Radiology,2023
2820,Vyaw23,mastersthesis,Automated segmentation of subsolid pulmonary n...,"Vyawahare, Sanyog and Venkadesh, Kiran Vaidhya...",None,None,Master thesis,2023
2821,Phil23,mastersthesis,Body Composition Assessment in 3D CT Images,"Philipp, Lena",None,None,Master thesis,2023


Used for title matching

In [24]:
df_bib_data_with_titles = df_bib_data[df_bib_data['title'].notna()]
df_bib_data_with_titles.head()

,bibkey,type,title,authors,doi,gs_citations,journal,year
1069,Abas05,article,Analysis of regularity in the {EEG} background...,D. Ab\'{a}solo and R. Hornero and P. Espino an...,10.1016/j.clinph.2005.04.001,233,CLINNP,2005
1070,Genu22,conference,Imaging tumor-infiltrating CD8 (+) T-cells in ...,E. A. J. van Genugten and B. Piet and G. Schre...,None,None,None,2022
1071,Abas05a,inproceedings,EEG} background activity analysis in {A}lzheim...,D. Ab\'{a}solo and C. G\'{o}mez and J. Poza an...,None,3,None,2005
1072,Abel19,article,"Computational pathology definitions, best prac...","Abels, Esther and Pantanowitz, Liron and Aeffn...",10.1002/path.5331,24,JPAT,2019
1073,Abra08a,article,Evaluation of a system for automatic detection...,M. D. Abr\`amoff and M. Niemeijer and M. S. A....,10.2337/dc07-1312,287,DC,2008


# Gather ss and bib data for all staff member IDs

In [25]:
staff_id_dict = {'Bram van Ginneken': [8038506, 123637526],
'Francesco Ciompi': [143613202],
'Alessa Hering': [153744566],
'Henjan Huisman': [34754023],
'Colin Jacobs': [2895994],
'Peter Koopmans': [34726383],
'Jeroen van der Laak': [145441238, 145388932],
'Geert Litjens': [145959882],
'James Meakin': [4960344],
'Keelin Murphy': [35730362],
'Ajay Patel': [2109170880, 2116215861],
'Cornelia Schaefer-Prokop': [1419819133, 1445069528, 1400632685],
'Matthieu Rutten': [2074975080, 2156546],
'Jos Thannhauser': [5752941],
"Bram Platel" : [1798137], 
"Nico Karssemeijer" : [1745574], 
"Clarisa Sanchez" : [144085811, 32187701], 
"Nikolas Lessman" : [2913408], 
"Jonas Teuwen" : [32649341, 119024451], 
"Rashindra Manniesing" : [2657081]}

In [26]:
staff_year_dict = {
'Bram van Ginneken':  {'start' : 1996, 'end': 9999},
'Francesco Ciompi':  {'start' : 2013, 'end': 9999},
'Alessa Hering':  {'start' : 2018, 'end': 9999},
'Henjan Huisman':  {'start' : 1992, 'end': 9999},
'Colin Jacobs':  {'start' : 2010, 'end': 9999},
'Peter Koopmans':  {'start' : 2022, 'end': 9999},
'Jeroen van der Laak':  {'start' : 1991, 'end': 9999},
'Geert Litjens':  {'start' : 2016, 'end': 9999},
'James Meakin':  {'start' : 2017, 'end': 9999},
'Keelin Murphy':  {'start' : 2018, 'end': 9999},
'Ajay Patel':  {'start' : 2015, 'end': 9999},
'Cornelia Schaefer-Prokop':  {'start' : 2010, 'end': 9999},
'Matthieu Rutten':  {'start' : 2019, 'end': 9999},
'Jos Thannhauser': {'start' : 2022, 'end': 9999},
"Bram Platel" : {'start' : 2010,  'end' : 2019},
"Nico Karssemeijer" : {'start' : 1989, 'end' : 2022}, 
"Clarisa Sanchez" : {'start' : 2008, 'end' : 2021}, 
"Nikolas Lessman" : {'start' : 2019, 'end' : 2022}, 
"Jonas Teuwen" : {'start' : 2017, 'end' : 2020}, 
"Rashindra Manniesing" : {'start' : 2010, 'end' : 2021}
}

In [27]:
staff_dict = {key: {'ids': staff_id_dict[key], 'years': staff_year_dict[key]} for key in staff_id_dict}
staff_dict

{'Bram van Ginneken': {'ids': [8038506, 123637526],
  'years': {'start': 1996, 'end': 9999}},
 'Francesco Ciompi': {'ids': [143613202],
  'years': {'start': 2013, 'end': 9999}},
 'Alessa Hering': {'ids': [153744566], 'years': {'start': 2018, 'end': 9999}},
 'Henjan Huisman': {'ids': [34754023], 'years': {'start': 1992, 'end': 9999}},
 'Colin Jacobs': {'ids': [2895994], 'years': {'start': 2010, 'end': 9999}},
 'Peter Koopmans': {'ids': [34726383], 'years': {'start': 2022, 'end': 9999}},
 'Jeroen van der Laak': {'ids': [145441238, 145388932],
  'years': {'start': 1991, 'end': 9999}},
 'Geert Litjens': {'ids': [145959882], 'years': {'start': 2016, 'end': 9999}},
 'James Meakin': {'ids': [4960344], 'years': {'start': 2017, 'end': 9999}},
 'Keelin Murphy': {'ids': [35730362], 'years': {'start': 2018, 'end': 9999}},
 'Ajay Patel': {'ids': [2109170880, 2116215861],
  'years': {'start': 2015, 'end': 9999}},
 'Cornelia Schaefer-Prokop': {'ids': [1419819133, 1445069528],
  'years': {'start': 201

In [28]:
# staff_ids = []
# [staff_ids.extend(values) for values in staff_dict.values()];
# staff_ids

## Semantic scolar

In [35]:
all_staff_id_ss_data = []

# get data for staff memb|er
for idx, (staff_name, values) in enumerate(staff_dict.items()):
    staff_ids = values['ids']
    staff_start = values['years']['start']
    staff_end = values['years']['end']
    print(f'[{idx+1}/{len(staff_dict.items())}]: {staff_name}')
    for staff_id in staff_ids:
        print('\t\t', staff_id)
        staff_id_ss_data = []
        # get semantic scolar data
        url=f'https://api.semanticscholar.org/graph/v1/author/{staff_id}/papers?fields=year,title,authors,externalIds,citationCount&limit=500'
        r = requests.get(url)
        ss_staff_data = r.json()['data']

        # loop over semantic scolar data entries
        for ss_staff_entry in ss_staff_data:
            ss_id = None
            ss_title = None
            ss_doi = None
            ss_citations = None
            ss_year = None

            if 'paperId' in ss_staff_entry:
                ss_id = ss_staff_entry['paperId']
            if 'title' in ss_staff_entry:
                ss_title = ss_staff_entry['title']
            if 'DOI' in ss_staff_entry['externalIds']:
                ss_doi = ss_staff_entry['externalIds']['DOI']
            if 'citationCount' in ss_staff_entry:
                ss_citations = ss_staff_entry['citationCount']
            if 'year' in ss_staff_entry:
                ss_year = ss_staff_entry['year']

            if ss_year != None:
                if not staff_start <= ss_year <= staff_end:
                # probably doesnt belong to DIAG, still captured via another staff member if also in the same paper
                    continue
                
            staff_id_ss_data.append([staff_id, staff_name, staff_start, staff_end, ss_year, ss_id, ss_title, ss_doi, ss_citations])
        all_staff_id_ss_data.extend(staff_id_ss_data)    
ss_columns = ['staff_id', 'staff_name', 'staff_from', 'staff_till', 'ss_year', 'ss_id', 'title', 'doi', 'ss_citations']
df_all_staff_id_ss_data = pd.DataFrame(all_staff_id_ss_data, columns=ss_columns)
print('DONE')

[1/20]: Bram van Ginneken
		 8038506
		 123637526
[2/20]: Francesco Ciompi
		 143613202
[3/20]: Alessa Hering
		 153744566
[4/20]: Henjan Huisman
		 34754023
[5/20]: Colin Jacobs
		 2895994
[6/20]: Peter Koopmans
		 34726383
[7/20]: Jeroen van der Laak
		 145441238
		 145388932
[8/20]: Geert Litjens
		 145959882
[9/20]: James Meakin
		 4960344
[10/20]: Keelin Murphy
		 35730362
[11/20]: Ajay Patel
		 2109170880
		 2116215861
[12/20]: Cornelia Schaefer-Prokop
		 1419819133
		 1445069528
[13/20]: Matthieu Rutten
		 2074975080
		 2156546
[14/20]: Jos Thannhauser
		 5752941
[15/20]: Bram Platel
		 1798137
[16/20]: Nico Karssemeijer
		 1745574
[17/20]: Clarisa Sanchez
		 144085811
[18/20]: Nikolas Lessman
		 2913408
[19/20]: Jonas Teuwen
		 32649341
[20/20]: Rashindra Manniesing
		 2657081
DONE


In [36]:
df_all_staff_id_ss_data = df_all_staff_id_ss_data.sort_values('ss_id').reset_index(drop=True)

In [37]:
df_all_staff_id_ss_data.to_csv('script_data/temp_semantic_scholar_all_staff_id_last_500_summary.csv', index=False)

# Loop for matching DOIs and titles

In [38]:
from collections import Counter

In [39]:
doi_match_data = []
match_keys_columns = ['bibkey', 'ss_id', 'all_ss_ids', 'comment']
print('[MATCHING ON DOIS]')
# LOOP OVER BIB
for idx_bib, row_bib in df_bib_data.iterrows():
    bibkey = row_bib['bibkey']
    comment = None
    all_ss_ids = None
    
    bib_doi = row_bib['doi']
    if bib_doi == None:
        continue
    index = bib_doi.find('10.')
    bib_doi = bib_doi[index:] # remove url leading to doi identifier 
    doi_matching = df_all_staff_id_ss_data['doi'].isin([bib_doi])
    
    if doi_matching.any():
        matches = df_all_staff_id_ss_data[doi_matching] # could be multiple in SS, search for "Epithelium segmentation using deep learning" paper from Wouter Bulten... 
        if len(Counter(matches['ss_id'])) == 1:
            comment = 'single doi match'
            match = matches.iloc[0]
        if len(Counter(matches['ss_id'])) > 1:
            all_ss_ids = list(Counter(matches["ss_id"]).keys())
            comment = f'multiple doi matches {all_ss_ids}, took the first one with most citations'
            match = matches[matches['ss_citations'] == matches['ss_citations'].max()].iloc[0]
        ss_id = match['ss_id']
        doi_match_data.append([bibkey, ss_id, all_ss_ids, comment])

df_doi_match_data = pd.DataFrame(doi_match_data, columns = match_keys_columns)
df_doi_match_data.to_csv('script_data/temp_doi_match_data.csv', index=False)
print('DONE')
print(len(df_doi_match_data['comment'].notna()))
print(len(df_doi_match_data))

[MATCHING ON DOIS]
DONE
837
837


In [40]:
np.unique(df_doi_match_data['comment'], return_counts=True)[1]

array([  1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,
         1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,
       811], dtype=int64)

In [41]:
import time

In [43]:
start = time.time()
print('[MATCHING ON TITLES]')
title_match_data = []
for idx_ss, row_ss in df_all_staff_id_ss_data.iterrows():
    if idx_ss % 50 == 0:
        print(f'[{idx_ss}/{len(df_all_staff_id_ss_data)}]')
    
    ss_id = row_ss['ss_id']
    # if ss_id in df_doi_match_data['ss_id']:
    #     continue
 
    ss_title = row_ss['title']
    if pd.isna(ss_title):
        continue
    
    title_match_ratios = df_bib_data_with_titles['title'].apply(lambda x: SequenceMatcher(
        a=ss_title.lower(), 
        b=x.lower().replace('{', '').replace('}', '')).ratio())

    max_ratio = title_match_ratios.max()
    max_bibkey = df_bib_data_with_titles[title_match_ratios==max_ratio]['bibkey'].iloc[0]
    max_bib_title = df_bib_data_with_titles[title_match_ratios==max_ratio]['title'].iloc[0]
    max_bib_title = max_bib_title.replace('{', '').replace('}', '')
    
    up80_bibkeys = None
    up80_bib_titles = None
    up80_ratios = None
    if sum(title_match_ratios>0.8) > 1:
        
        up80_bibkeys = list(df_bib_data_with_titles[title_match_ratios>0.8]['bibkey'])
        up80_bib_titles = list(df_bib_data_with_titles[title_match_ratios>0.8]['title'])
        up80_bib_titles = [up80_bib_title.replace('{', '').replace('}', '') for up80_bib_title in up80_bib_titles]
        up80_ratios = list(title_match_ratios[title_match_ratios>0.8])
    
    title_match_data.append([ss_id, ss_title, 
                             max_bibkey, max_bib_title, max_ratio, 
                             up80_bibkeys, up80_bib_titles, up80_ratios])

title_matches_columns = ['ss_id', 'ss_title', 
                         'max_bibkey' , 'max_bib_title', 'max_ratio',
                         'up80_bibkeys' , 'up80_bib_titles', 'up80_ratios',
                        ]
df_title_match_data = pd.DataFrame(title_match_data, columns=title_matches_columns)
df_title_match_data.to_csv('script_data/temp_title_match_data.csv', index=False)
print('DONE')
end = time.time()
sec = end-start
print(sec, 'sec')
print(sec/60, 'min')

[MATCHING ON TITLES]
[0/2183]
[50/2183]
[100/2183]
[150/2183]
[200/2183]
[250/2183]
[300/2183]
[350/2183]
[400/2183]
[450/2183]
[500/2183]
[550/2183]
[600/2183]
[650/2183]
[700/2183]
[750/2183]
[800/2183]
[850/2183]
[900/2183]
[950/2183]
[1000/2183]
[1050/2183]
[1100/2183]
[1150/2183]
[1200/2183]
[1250/2183]
[1300/2183]
[1350/2183]
[1400/2183]
[1450/2183]
[1500/2183]
[1550/2183]
[1600/2183]
[1650/2183]
[1700/2183]
[1750/2183]
[1800/2183]
[1850/2183]
[1900/2183]
[1950/2183]
[2000/2183]
[2050/2183]
[2100/2183]
[2150/2183]
DONE
1386.101304769516 sec
23.101688412825265 min


In [ ]:
print('DONE WITH THIS NOTEBOOK')
1/0

# DONE